In [ ]:
# Install necessary libraries
!pip install pandas numpy prophet matplotlib scikit-learn
!pip install statsforecast datasetsforecast

In [ ]:
# Step 1: Import the necessary libraries
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import warnings
import itertools
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
from statsforecast import StatsForecast
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.style.use('grayscale') # fivethirtyeight  grayscale  classic
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#008080',  # #212946
    'axes.facecolor': '#008080',
    'savefig.facecolor': '#008080',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#000000',  #2A3459
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

from pylab import rcParams
rcParams['figure.figsize'] = (18,7)
warnings.filterwarnings('ignore')

In [ ]:
# Step 2: Load the main dataset and preprocess
data_path = '/kaggle/input/cmu-dsproject2024/Data_withoutBillNo_clean.csv'
df_0 = pd.read_csv(data_path, low_memory=False)
print(df_0.info())
df_0.head(1)

In [ ]:
# Step 2.1: Data preprocess for ITEM_CODE group
df = df_0.copy()
# Replace '.' with '-'
df['ITEM_CODE'] = df['ITEM_CODE'].str.replace('.', '-')
# Remove leading and trailing spaces
df['ITEM_CODE'] = df['ITEM_CODE'].str.strip()
# DataFrame named 'df' with an 'ITEM_CODE' column
df['ITEM_CODE'] = df['ITEM_CODE'].astype(str)  # Convert to string
# Replace values starting with '10203--01' with '10203-01'
df['ITEM_CODE'] = df['ITEM_CODE'].str.replace('10203--01', '10203-01')
df['ITEM_CODE'].head(1)

# Convert 'BILL_DATE' to datetime and filter necessary columns for analysis
df['BILL_DATE'] = pd.to_datetime(df['BILL_DATE'], format='%Y%m%d')
# Convert 'QTY' to strings and then to numeric, handling commas and non-numeric characters
df.loc[:, 'QTY'] = pd.to_numeric(df['QTY'].astype(str).str.replace(',', ''), errors='coerce')
# Aggregate the data by BILL_DATE and ITEM_CODE, summing QTY and pivot each ITEM_CODE as a separate column
df = df.groupby(['BILL_DATE', 'ITEM_CODE'])['QTY'].sum().unstack(fill_value=0).reset_index() 
# Ensure that the 'BILL_DATE' is renamed to 'ds' (required by Prophet)
df.rename(columns={'BILL_DATE': 'ds'}, inplace=True)

#Calculate the sum of columns 2 to end (item codes)
# Select only numeric columns (assuming item codes are numeric)
numeric_cols = [col for col in df.columns if col not in ['ds', 'ITEM_CODE']]  # Exclude 'ds' and 'ITEM_CODE'
# Calculate the sum of numeric columns
df['y'] = df[numeric_cols].sum(axis=1)
df['unique_id'] = 1
df.head(1)

In [ ]:
# Step 2.2: Data preprocess for REG_Code group
df_reg = df_0.copy()
# Remove leading and trailing spaces
df_reg['REG_Code'] = df_reg['REG_Code'].str.strip()
# DataFrame named 'df' with an 'REG_Code' column
df_reg['REG_Code'] = df_reg['REG_Code'].astype(str)  # Convert to string
# Convert 'BILL_DATE' to datetime and filter necessary columns for analysis
df_reg['BILL_DATE'] = pd.to_datetime(df_reg['BILL_DATE'], format='%Y%m%d')
# Convert 'QTY' to strings and then to numeric, handling commas and non-numeric characters
df_reg.loc[:, 'QTY'] = pd.to_numeric(df_reg['QTY'].astype(str).str.replace(',', ''), errors='coerce')
# Aggregate the data by BILL_DATE and REG_Code, summing QTY and pivot each REG_Code as a separate column
df_reg = df_reg.groupby(['BILL_DATE', 'REG_Code'])['QTY'].sum().unstack(fill_value=0).reset_index() 
# Ensure that the 'BILL_DATE' is renamed to 'ds' (required by Prophet)
df_reg.rename(columns={'BILL_DATE': 'ds'}, inplace=True)
df_reg.head(1)

In [ ]:
# Step 2.3: Data preprocess for DEST_CODE group
df_dest = df_0.copy()
# Remove leading and trailing spaces
df_dest['DEST_CODE'] = df_dest['DEST_CODE'].str.strip()
# DataFrame named 'df' with an 'DEST_CODE' column
df_dest['DEST_CODE'] = df_dest['DEST_CODE'].astype(str)  # Convert to string
# Convert 'BILL_DATE' to datetime and filter necessary columns for analysis
df_dest['BILL_DATE'] = pd.to_datetime(df_dest['BILL_DATE'], format='%Y%m%d')
# Convert 'QTY' to strings and then to numeric, handling commas and non-numeric characters
df_dest.loc[:, 'QTY'] = pd.to_numeric(df_dest['QTY'].astype(str).str.replace(',', ''), errors='coerce')
# Aggregate the data by BILL_DATE and DEST_CODE, summing QTY and pivot each DEST_CODE as a separate column
df_dest = df_dest.groupby(['BILL_DATE', 'DEST_CODE'])['QTY'].sum().unstack(fill_value=0).reset_index() 
# Ensure that the 'BILL_DATE' is renamed to 'ds' (required by Prophet)
df_dest.rename(columns={'BILL_DATE': 'ds'}, inplace=True)
df_dest.head(1)

In [ ]:
# Step 3: EDA for Stationarity check, Autocorrelation and Seasonal Decompose plot
# Step 3.1: EDA of ITEM_Code group
# Check the data type of 'y' column
if df['y'].dtypes == 'object':
    # Try converting the column to numeric (assuming it contains numbers as strings)
    try:
        df['y'] = pd.to_numeric(df['y'])
    except:
        # Handle the case where conversion fails (e.g., non-numeric strings)
        print("Error: Could not convert 'y' column to numeric. Please clean the data.")

StatsForecast.plot(df)

In [ ]:
# Step 3.2: Stationarity check by ADF test
def Augmented_Dickey_Fuller_Test_func(series , column_name):
    print (f'Dickey-Fuller test results for columns: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of observations used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("The data is stationary")
    else:
        print("Conclusion:====>")
        print("The null hypothesis cannot be rejected")
        print("The data is not stationary")
        
Augmented_Dickey_Fuller_Test_func(df['y'],"Life expectancy")

In [ ]:
# Step 3.3: Plot Autocorrelation and Partial Autocorrelation
fig, axs = plt.subplots(nrows=1, ncols=2)
plot_acf(df['y'],  lags=30, ax=axs[0],color="fuchsia")
axs[0].set_title("Autocorrelation of Total Product");
# Grafico
plot_pacf(df['y'],  lags=30, ax=axs[1],color="lime")
axs[1].set_title('Partial Autocorrelationof Total Product')
plt.show();

In [ ]:
#Step 3.4: Defind and plot Seasonal Decompose
def plotSeasonalDecompose(
    x,
    model='additive',
    filt=None,
    period=None,
    two_sided=True,
    extrapolate_trend=0,
    title="Seasonal Decomposition"):
    result = seasonal_decompose(
            x, model=model, filt=filt, period=period,
            two_sided=two_sided, extrapolate_trend=extrapolate_trend)
    fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"])
    for idx, col in enumerate(['observed', 'trend', 'seasonal', 'resid']):
        fig.add_trace(
            go.Scatter(x=result.observed.index, y=getattr(result, col), mode='lines'),
                row=idx+1, col=1,
            )
    return fig

plotSeasonalDecompose(
    df['y'],
    model="additive",
    period=28,
    title="Seasonal Decomposition")

In [ ]:
# Step 4: Feature engineerin data handelig for feed model
# Load the external factor dataset
external_df = pd.read_csv('/kaggle/input/cmu-ds2024-ext-factor/Data_withoutBillNo_ext_fector.csv',  low_memory=False )
external_df['ds'] = pd.to_datetime(external_df['ds'])
external_df.rename(columns={'y': 'Total'}, inplace=True)

# Extrect trend and seasonal by MSTL Decompose function
# Define seasonal periods (28)
seasonal_periods = [28]
def mstl_decompose(series, seasonal_periods):
    """
    Perform Multiple Seasonal-Trend decomposition using STL.
    Args:
        series (pd.Series): Time series data.
        seasonal_periods (list): List of seasonal periods to decompose.
    Returns:
        dict: Decomposed components including trend, each seasonal component, and residual.
    """
    residual = series.copy()
    components = {}

    for i, period in enumerate(seasonal_periods):
        stl = STL(residual, period=period, robust=True)
        result = stl.fit()
        seasonal = result.seasonal
        trend = result.trend
        resid = result.resid
        components[f'seasonal_{i+1}'] = seasonal
        residual = resid  # Update residual for next seasonality
    
    # The final residual after removing all seasonal components
    components['trend'] = trend
    components['resid'] = residual
    return components

# MSTL to all product series
mstl_components = {}
for col in external_df.columns:
    if col.startswith('Total'):
        mstl_components[col] = mstl_decompose(external_df[col], seasonal_periods)
        external_df['trend'] = mstl_components[col]['trend']
        external_df['seasonal'] = sum([mstl_components[col][f'seasonal_{i+1}'] for i in range(len(seasonal_periods))])
external_df.head(1)


In [ ]:
#External factor select columns
external_df = external_df[['ds', 'gasohol_95', 'trend', 'seasonal']]
# Select relevant columns and perform normalization
for col in ['gasohol_95', 'trend', 'seasonal']:
    scaler = MinMaxScaler(feature_range=(0, 1))
    external_df[f"{col}_normalized"] = scaler.fit_transform(external_df[[col]])

df.rename(columns={'y': 'Total'}, inplace=True)
# Drop the 'unique_id' column from df (if intended)
df = df.drop(columns=['unique_id'])
# Merge df and df_reg on 'ds' column with left join
df = pd.merge(df, df_reg, on='ds', how='left')
# Merge df and df_dest on 'ds' column with left join
df = pd.merge(df, df_dest, on='ds', how='left')
# Merge external factors
df = pd.merge(df, external_df[['ds'] + [col for col in external_df if col.endswith('normalized')]], on='ds', how='left')

df.head(1)

In [ ]:
input_df = df.to_csv('/kaggle/working/input_df.csv', index=True)

In [ ]:
# Update items_list
items_list = df.columns[1:-4].tolist()  # Exclude 'ds', 'Total', normalized external factors

# Split the data into train and validation sets
train_data = df[(df['ds'] >= '2024-01-01') & (df['ds'] <= '2024-05-31')]
val_test_data = df[(df['ds'] >= '2024-06-01') & (df['ds'] <= '2024-06-30')]
print('val_test_data.shape', val_test_data.shape)
val_test_data.head(1)

In [ ]:
# Step 5: Build the Prophet model for each product and forecast
models = []
forecasts = []

# Fit Prophet models with external factors as regressors
for item in items_list:
    temp_df = train_data[['ds', item, 'gasohol_95_normalized', 'trend_normalized', 'seasonal_normalized']].rename(columns={item: 'y'})  # Prepare data for Prophet
    
    # Initialize the Prophet model and add external factors as regressors
    model = Prophet()
    for col in external_df.columns:  # Add all normalized external factors
        if col.endswith('normalized'):
            model.add_regressor(col)
    
    # Fit the model
    model.fit(temp_df)
    
    # Create future dataframe for forecast (June 2024)
    future = model.make_future_dataframe(periods=len(val_test_data), freq='D')
    for col in external_df.columns:  # Set future values for external factors
        if col.endswith('normalized'):
            future[col] = external_df.set_index('ds').loc[future['ds'], col].values
    
    # Forecast
    forecast = model.predict(future)
    
    # Clip the forecast to ensure no negative values (since quantity must be >= 0)
    forecast['yhat'] = np.clip(forecast['yhat'], a_min=0, a_max=None)
    
    forecasts.append(forecast[['ds', 'yhat']])
    
    # Append model to list
    models.append(model)

# Combine all the forecasts into a single DataFrame
combined_forecasts = pd.concat([forecast.set_index('ds')['yhat'] for forecast in forecasts], axis=1)
combined_forecasts.columns = items_list

# Step 6: Evaluate the model using RMSE
forecast_val_test_period = combined_forecasts.loc[val_test_data['ds']]
rmse_scores = {}

# Calculate RMSE for each product in validation period
for item in items_list:
    if item in val_test_data.columns:
        y_true = val_test_data[item]
        y_pred = forecast_val_test_period[item]
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        rmse_scores[item] = rmse

# Calculate total RMSE
total_rmse = np.mean(list(rmse_scores.values()))
print(f"Total RMSE for the ProphetBestModel_item_reg_dest: {total_rmse:.4f}")


In [ ]:
# Step 7: Save the forecasted results
# Save the forecast to a CSV file
combined_forecasts.to_csv('/kaggle/working/submit_cmu_ds_ProphetBestModel_item_reg_dest.csv', index=True)
print("Forecast saved to 'submit_cmu_ds_ProphetBestModel_item_reg_dest.csv'.")

In [ ]:
# Step 8: Visualization of Forecasts vs Actuals
selected_series = ['00000-00', '99999-000', '99999-006', 'R30', 'D32']

# Plot actual vs forecasted values for selected series
for series in selected_series:
    if series not in train_data.columns:
        print(f"Column '{series}' not found in training data. Skipping.")
        continue
    if series not in val_test_data.columns:
        print(f"Column '{series}' not found in validation data. Skipping.")
        continue

    # Plot the data if the column exists in both datasets
    plt.figure(figsize=(12, 6))
    plt.plot(train_data['ds'], train_data[series], label='Training Data')
    plt.plot(val_test_data['ds'], val_test_data[series], label='Actual')
    plt.plot(combined_forecasts.index, combined_forecasts[series], label='Forecast')

    plt.title(f"Forecast QTY for CODE {series} by ProphetBestModel")
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [ ]:
# Step 9: Analyze RMSE Distribution
import seaborn as sns

rmse_df = pd.DataFrame.from_dict(rmse_scores, orient='index', columns=['RMSE'])
plt.figure(figsize=(10, 6))
sns.histplot(rmse_df['RMSE'], bins=50, kde=True)
plt.title('Distribution of RMSE Scores Across QTY of Products by ProphetBestModel')
plt.xlabel('RMSE')
plt.ylabel('Frequency')
plt.show()

# Save RMSE scores to CSV
rmse_df.to_csv('/kaggle/working/rmse_scores_ProphetBestModel_item_reg_dest.csv')
print("RMSE scores saved to 'rmse_scores_ProphetBestModel_item_reg_dest.csv'.")